# Mobile Robotics Project 
## Groupe 47

| Name      | SCIPER  |
|-----------|---------|
| Martin    | 340936  |
| Laetitia  | 325743  |
| Arthur    | 300443  |
| Amandine  | 344736  |

A **Jupyter notebook** which serves as a report. This must contain the information regarding :
- The **members of the group**, it’s helpful to know who you are when we go over the report.
- An **introduction to your environment** and to the **choices you made**.
- Sections that go into a bit **more detail regarding the implementation** and are accompanied by the code required to **execute the different modules independently**. What is important is not to simply describe the code, which should be readable, but describe what is not in the code: the theory behind, the choices made, the measurements, the choice of parameters etc. Do not forget to **cite your sources** ! You can of course show how certain modules work in **simulation here, or with pre-recorded data**.
- A section which is used to **run the overall project** and where we can see the path chosen, where the system believes the robot is along the path before and after filtering etc… This can also be done **in a .py file** if you prefer. Just make sure to reference it in the report so we can easily have a look at it.
- Conclusion: The code used to execute the overall project. Do not hesitate to **make use of .py files** that you import in the notebook. The whole body of code does not have to be in the Jupyter notebook, on the contrary! Make sure the **code clean and well documented**. Somebody who starts to browse through it should easily understand what you are doing.

# Introduction
As part of the course Basics of mobile robotics, given to master students at EPFL by professor Mondada, we did a project using a Thymio and a camera. First, with computer vision, we map the environment where the Thymio will evolve, creating a grid based on the limits of the workspace and mapping global obstacles and the goal the robot has to reach. We localize the Thymio and find the shortest path he can follow to reach the goal without encountering any global obstacles. The Thymio then follows this path and can react to local obstcles added in front of him, avoiding them and continuing towards his goal. 

In our project the robot is moving on a white A0 sheet for better contrasts and flatness with 4 aruco markers on the borders to better correct the perspective. The Thymio has an aruco markers on him for the detection. The global obstacles are red paper sheets to make arbitrary shapes and move them around easily. For the same reasons, the goal is a green paper sheet. The local obstacles are represented as 5cm circular paprika container.

## The Arena
- Background: a white A0 sheet
- Goal: a green shape
- Global obstacles: red shapes (seen by the camera)
- Local obstacles (invisible to camera): circular object of diameter $\approx 5cm$, like a paprika container
- Aukey PC-W1 Webcam 1080p
- Wireless Thymio with Aruco {id:9, size:70mm}
- Top Left ArUco: {id:0, size:70mm}
- Bottom Left ArUco: {id:1, size:70mm}
- Top Right ArUco: {id:10, size:70mm}
- Bottom Right ArUco: {id:2, size:70mm}

<img src="https://i.postimg.cc/CLs2grqx/image-2024-12-04-145126355.png" alt="env setup" width="500">

Note: the aruco markers placement order and orientation is very important for computer vision.

# Implementation
Here is a flow chart of the code structure. The code is divided in 4 files. The main file following the flow chart, the Thymio class containing the thymio's information, the thymio's detection, the kalman filter and the two controllers (Astolfi and local avoidance), the camera class containing the camera warmup, image capture, arena orner detection, perspective correction, obstacle and goal detection and plotting. The 4th file is for the path planning (A*, key points image to grid conversion)
We will then go through these features:
1. Computer vision : arena corners, Thymio, obstacles and goal detections, obstacles expansion
2. Path Planning : A* Algorithm, key points
3. Kalman Filter : Motion equations and covariances measurements
4. Motion control : Astolfi Controller
5. Local Avoidance : Braitenberg

And finally we will see how they are integrated in the main programm.

<img src="https://i.postimg.cc/yNG1T81t/image-2024-12-04-151247433.png" alt="flowchart" width="700">

## 1. Computer vision
This section will present the computer vision implementation. We used two different detection methods: The aruco markers and the color detection.

As the camera has automatic white balance (AWB) and automatic exposure (AE), the very first frames appear yellowish. To give the camera time to adapt to the scene, we take 50 images:
```python
for _ in range(50):  # Camera needs to do lighting and white balance adjustments
    ret, _ = self.cam.read()
    if not ret:
        self.cam.release()
        raise IOError("Failed to capture frame during warmup")
```

### 1.a ArUco Markers
We choose aruco markers for detecting the corners of the arena (the environement containing the robot, obstacles and goal) and the thymio itself, because they are robust to varying lighting conditions and provide precise detection. We used the original aruco dictionnary as it proved to be more robust to us after trying multiple dictionnaries. 
We choose to use the OpenCV aruco detector module for a more straightforward,efficient and robust approach. The steps of the detection can be found in [the documentation](https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html) and [here](https://mmla.gse.harvard.edu/tools/js-aruco-marker-detector/). To get the markers, we used the [ArUco markers generator!](https://chev.me/arucogen/) by Oleg Kalachev. We chose the markers to have a size of $70mm$ and ids: $[0,1,2,10]$ for the arena and $9$ for the Thymio.



#### 1.a.i Corner detection and perspective
`find_aruco_corners_size()`
After getting the aruco markers in the corners of the arena we want to get a conversion factor from pixels to mm to later convert the Thymio's position in pixel to mm for the Kalman filter and controller. We simply take the mean of all 16 sides of the markers and divide it by the real length in mm:
```python
size_aruco_px=[]
for i in range(4):
    side_lengths = [
        np.linalg.norm(corners[i][0,0,:] - corners[i][0,1,:]),  # Top side
        np.linalg.norm(corners[i][0,1,:] - corners[i][0,2,:]),  # Right side
        np.linalg.norm(corners[i][0,2,:] - corners[i][0,3,:]),  # Bottom side
        np.linalg.norm(corners[i][0,3,:] - corners[i][0,0,:])   # Left side
    ]
    size_aruco_px.append(np.mean(side_lengths))
return np.array(outer_corners), np.mean(size_aruco_px)
```
Having the corners of the A0 sheet, we find the maximum width and height of the area defined by the 4 corners, and then compute the perspective transformation matrix $M$, that maps from the distroted space to the new perspective corrected space. Finally, we apply this transformation by using `cv2.warpPerspective()` to isolate our arena, which allows better visualisation and detection. This is necessary, as our camera is not exactly above the A0 sheet and perfectly centered ("trapezoidal" deformation), and it makes the edges of the A0 sheet (x and y axis of our coordinate system) parallel to the edges of the corrected image frame (pixel coordinate system), aswell as to find the convertion ratio from mm to pixel, needed later for Kalman:
```python
corners, size_aruco_px = find_aruco_corners_size(image)
ordered_corners = order_points(corners)
M = cv2.getPerspectiveTransform(ordered_corners, destination_corners)
max_width_perspective, max_height_perspective = compute_destination_size(ordered_corners)
persp_image = cv2.warpPerspective(image, M, (max_width_perspective, max_height_perspective), flags=cv2.INTER_LINEAR)
pixbymm = size_aruco_px / corner_aruco_size_mm
```

Note that the computation is $M$ only done once, at the initialization.

#### 1.a.ii Thymio detection
`Thymio_position_aruco()`
Similarly, the Thymio's position is found with OpenCV's aruco marker detector, maker id = 9. By doing the mean of the marker's corners, we find $(x,y)$.
```python
idx = np.where(ids == self.Thymio_ID)[0][0] #Thymio's aruco ID is 9
aruco_corners=np.array(corners[idx][0,:,:])
Thymio_x,Thymio_y=aruco_corners.mean(axis=0)
```
To find the angle $\theta$ we take the mean of the angle of the top and bottom edge of the marker with reference to the x axis:
```python
top_edge=aruco_corners[1,:]-aruco_corners[0,:]
bottom_edge=aruco_corners[2,:]-aruco_corners[3,:]
angle = np.mean([np.arctan2(bottom_edge[1], bottom_edge[0]), 
                np.arctan2(top_edge[1], top_edge[0])])
```

#### 1.b Color detection
The detection of the obstacle and goal is done in multiple steps in `full_detection_cnt_centroid()`:
1. We create a binary mask with color thresholding. We pick the color thresholds by using an interactive thresholder to find $[{(B_{low}, G_{low}, R_{low})_{green}, (B_{high}, G_{high}, R_{high})_{green}}]$ and $[{(B_{low}, G_{low}, R_{low})_{red}, (B_{high}, G_{high}, R_{high})_{red}}]$. Because we only detect 2 colors: green for the goal and red for the global obstacles, that are fairly distinct we could relax our treshold to have a robust system.
2. For the goal, we take the biggest contour that is in the goal thresholded mask, as there is only 1 green object. Smaller contours are noise.
```python
goal_mask = cv2.inRange(image, THRESHOLD_GOAL_LOW, THRESHOLD_GOAL_HIGH])
goal_contours, _ = cv2.findContours(goal_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
goal_contour = max(goal_contours, key=cv2.contourArea)
```
3. Once the goal isolated, we find its center, a necessary step for the path planning algorithm.
```python
M = cv2.moments((goal_mask * 255).astype(np.uint8))
Goal_x = int(M["m10"] / M["m00"])
Goal_y = int(M["m01"] / M["m00"])
Goal_center = np.array([Goal_x, Goal_y]).reshape(2, 1)
```
5. For the global obstacles, similarly based on color thresholds, we create a mask with all the global obstacles. Then we remove noise by using the `filter_small_blobs(obstacles_mask, blob_min_size)`, this function will remove connected components with a size smaller than `blob_min_size`, to filter out noise as our global obstacles have significant size. After, `fill_holes` finds the contour for each obstacle with `cv2.findContours()` and then fills them with `cv2.drawContours(thickness=cv2.FILLED)`, to have smooth and well defined obstacles.
```python
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.drawContours(filled_mask, contours, -1, 255, thickness=cv2.FILLED)
```
6. Finally we expand the obstacles with Thymio's radius and mm to pixel convertion ratio, by taking all pixels that are at a certain distance from the obstacle, because we use the L2 norm as heuristic in the path planning we use the same distance here This is a crucial step for path planing to avoid Thymio and global obstacles collision.
```python
radius = 0.38 * Thymio_radius_mm * corner_aruco_size_pix / corner_aruco_size_mm
distance = cv2.distanceTransform(~obstacle_mask, cv2.DIST_L2, 5)
expanded_obstacle_mask = (distance < radius) * 255
```

## 2. Kalman filter
### 2.a Motion Model
We model the robot's state as $\mathbf{x} = [x, y, \theta]^T$, where:
- $x$ and $y$ are the position coordinates in millimeters.
- $\theta$ is the orientation angle in radians.

The motion of the robot is governed by the kinematic equations for a differential drive robot:
$\begin{pmatrix}
x(t+\Delta t) =x(t) + v \cdot \cos( \theta + \frac{\omega \cdot \Delta t}{2}) \Delta t\\
y(t+\Delta t) = y(t) + v \cdot \sin(\theta + \frac{\omega \cdot \Delta t}{2}) \Delta t\\ 
\theta(t+\Delta t) = \theta(t) + \omega \cdot \Delta t
\end{pmatrix}$

where the input are the left and right motor speeds v_R and v_L that appears through $v = \frac{v_L + v_R}{2}$ and $\omega = \frac{v_L - v_R}{L}$


We do the Runga-Kutta 2 approximation for a better accuracy for a still simple and fast model as in this ["EKF lecture Princeton"](https://www.cs.princeton.edu/courses/archive/fall11/cos495/COS495-Lecture17-EKFLocalization.pdf)


## 2.b Kalman equations:
The Kalman filter equations used are the ones of the lecture on uncertainty. We have to use an extended Kalman filter because of the non-linear dynamics. For this we need to calculate the Jacobians of the motion model.
The equations are:
1. Prediction:
$x_t = g(u_t, x_{t-1})$ and
$\Sigma_t = G_t \Sigma_{t-1} G_t^\top + Q_t$
2. Update:
$K_t = \Sigma_t H_t^\top \left( H_t \Sigma_t H_t^\top + R_t \right)^{-1}$, $x_t = x_t + K_t \left( z_t - h(x_t) \right)$ and 
$\Sigma_t = \left( I - K_t H_t \right) \Sigma_t$

Where G and H are the Jacobians of g and h w.r.t x respectively.

However these equations asume Gaussian white noise on the process. However our process noise depends on the input. Indeed because there is no lateral movement if the robot goes in the x direction it's variance cannot be the same as in y. This is already partially taken into account in the equations with $G_t \Sigma_{t-1} G_t^\top$ but then it is hard to find Q if not by fine-tuning. We decided to calculate Q based on Q_control the covariance matrix of the control inputs as in ["EKF lecture Princeton"](https://www.cs.princeton.edu/courses/archive/fall11/cos495/COS495-Lecture17-EKFLocalization.pdf) or ["this paper"](https://www.sba.org.br/Proceedings/SBAI/SBAI2017/SBAI17/papers/paper_426.pdf):

We have $Q=G{u,t} \cdot Q_{control} \cdot G{u,t}^\top$ where $G{u,t}$ is the Jacobian of the motion model with respect to the inputs calculated as follows:
$$G_{u,t} =\begin{pmatrix}
\frac{\partial x}{\partial v_L} & \frac{\partial x}{\partial v_R} \\
\frac{\partial y}{\partial v_L} & \frac{\partial y}{\partial v_R} \\
\frac{\partial \theta}{\partial v_L} & \frac{\partial \theta}{\partial v_R}
\end{pmatrix}$$ 
giving
$$G_{u,t} =
\begin{pmatrix}
\frac{\Delta t}{2} \cos(\theta_{\text{mid}}) - \frac{(v_L + v_R) \sin(\theta_{\text{mid}}) \Delta t^2}{4L} & \frac{\Delta t}{2} \cos(\theta_{\text{mid}}) + \frac{(v_L + v_R) \sin(\theta_{\text{mid}}) \Delta t^2}{4L} \\
\frac{\Delta t}{2} \sin(\theta_{\text{mid}}) + \frac{(v_L + v_R) \cos(\theta_{\text{mid}}) \Delta t^2}{4L} & \frac{\Delta t}{2} \sin(\theta_{\text{mid}}) - \frac{(v_L + v_R) \cos(\theta_{\text{mid}}) \Delta t^2}{4L} \\
\frac{\Delta t}{L} & -\frac{\Delta t}{L}
\end{pmatrix}$$

where $\theta_{\text{mid}}=\theta + \frac{\omega \cdot \Delta t}{2}$ from RK2 as above.

## 2.c Covariance measurements
The covariances are key components in the Kalman Filter. To determine the different covariances matrices: Q of the process and R of the measurements we did the following:

### 2.c.i Process covariance Q
The process covariance represents the variance of the error due to the model. Errors can appear either because of our simplified model or because of wheel slippage etc. We had two different approaches to measure Q:

- First, as discussed before we directly used $Q$ in the predict step without the Jacobian $G_u$. For this we runned for different motor speeds and delta t's the Thymio and compared the predicted position and the measured with a precise ruler and a protractor.By assuming the covariance matrix was diagonal and $var(x)=var(y)$ we then computed the variance in $\frac{mm}{s}^2$ with `np.var((y_meas-y_pred)/dt)`. In the code we multiplied by $dt^2$ to get the covariance in $mm^2$.

- Secondly we tried with the method that seemed more correct and which is more widely supported by lecture notes and papers by taking into account the impact of the input on the motion through the Jacobian $G_u$. We then needed to find $Q_{control}$ because $Q=G{u,t} \cdot Q_{control} \cdot G{u,t}^\top$. We supposed $Q_{control}$ is diagonal and white noise because noise on speed measurement left and right are supposedly independent, gaussian. So we let the robot run for an amount of time dt at a certain target speed and we did the variance `np.var(v_L_meas-v_L_meas_target)` and the covariance becomes `Q_ctrl=np.diag([var(v_L_meas-v_L_meas_target),var(v_L_meas-v_R_meas_target)])` which is in Thymio speed (pwm) and so we multiply by the speed factor squared to get $\frac{mm}{s}^2$ and use it in our equations.


- SAY WHICH ONE WE USED WHY ETC

### 2.c.ii Measurements Covariance R
The covariance R accounts for the imprecision of the sensors that measures the states. In our case 


## 2.d Code implementation
In the code the prediction is done in the thymio class through `kalman_predict_state()`. It updates the estimated position (`xytheta_est`) to the predicted one. The update step is done with `kalman_update_state()`. It updates the estimated position (`xytheta_est`) to a fusion of the measurements and the predicted position.

If no Thymio is found, the update step is not done as there are no measurements in position but only in speed (the inputs)


# 3. Kidnapping

## 4. Path Planning
### 4.a A*
We choose the A star algorithm for path planning because:
- it is complete and will always give us a solution
- an image is like a grid so the preprocessing to get the grid is straightforward
- we can choose it's precision/running time by tuning the grid size: it is only done at the beginning (where we don't care about time) and at after each local avoidance so we can have a precise planning at the beginning and a faster after local avoidance
The downsides are:
- We choose to consider 8 neighbors and the L2 norm as heuristic because the robot can go diagonnaly. However, the robot is not limited to 45 degrees angles so the soltuion is not optimal for the robot
- we need to extract keypoints to give more latitude to the controller to o to each of them and to avoid too many changes:if the optimal is to go 30 degrees down right the path will be a lot of variations between going straight/down/45 degrees. With key points the robot only need to go from the beginning and end of the slope and could do 30 degrees directly optimizing a bit.

The implemenation of the A* is quite similar to the exercice session except a $\sqrt{2}$ to account for a longer distance when doing a diagonal move:
```python
if (neighbor[0]==current_pos[0]) or (neighbor[1]==current_pos[1]): #if goes straight
    tentative_g_cost = g_costs[current_pos]+(map_grid[neighbor]) #cost is 1 y default on the map_grid
else:
    tentative_g_cost = g_costs[current_pos]+(map_grid[neighbor])*np.sqrt(2) #going diagonnaly is going further

```


### 4.b Finding keypoints of the path

Given the size of the Thymio compared to the size of the grid cells, we won't follow the path cell by cell but instead extract keypoints that will serve as intermediate goals for the Thymio.
For cells every ```STEP``` cells, we study the vectors between the previous cell (```STEP``` cells before) and the current cell, and between the current cell and the next cell (```STEP``` cells after).
We calculate how much the robot has to turn to change from the first direction to the second one, and if this angle surpasses a threshold (meaning the turn is non negligable), we add the current cell to the list of keypoints of the path. Even if the angles are found insignificant, a cell is added to the keypoints every few steps to make sure we don't ignore too many small changes of direction that could become significant when added together.

```python
# direction vectors
dir_previous = (current[0] - previous[0], current[1] - previous[1])
dir_next = (next[0] - current[0], next[1] - current[1])

if (abs(find_rotation(dir_previous, dir_next)) > ANGLE_THRESHOLD): # significant change of direction
    keypoints.append(current)
    counter = 1
elif (counter >= COUNTER_THRESHOLD):  # ensures there isn't too much space between keypoints 
    keypoints.append(current)
    counter = 1
else:
    counter += 1
```

In this algorithm we chose to study the path in steps of size ```STEP``` instead of considering every single one of the cells because the cells of the grid are very small compared to the Thymio. 

Using ```math.atan2(det, dot_product)``` allows us to find the signed angle between the two vectors. atan2() can find angles over the whole trigonometric circle (using the information given by the sign of the dot_product), not limiting itself to only two quadrants like atan().

**Example : Calling ```find_keypoints(path)``` on a random path :**

```python
#parameters :
ANGLE_THRESHOLD = math.radians(20)
STEP = 2
COUNTER_THRESHOLD = 5 #max number of steps between keypoints
```

<img src="./images/path_example.jpg" alt="Diagram" width="200">


We created and displayed a grid with a random path to test the function `find_keypoints(path)`.

Here, we can see that most of the keypoints are added because of direction changes, except the fifth keypoint (in the middle of the third diagonal) that was added because of `COUNTER_THRESHOLD`. Indeed it is on the 10th cell after the previous keypoints (`COUNTER_THRESHOLD`*`STEP`=10).

# 5. Motion control

## 5.a Setting the motors' speed :

`limit_speed(v)` : limits the speed to 500, the maximum input accepted by the thymio

`set_motors(node, v_ml, v_mr)` : makes sure the speeds demanded do not exceed the maximum speed of 500, transforms the wanted speeds to int type (compatible with thymio), and then applies the adapted speeds to the motors.



## 5.b Controller : 

Diagram showing the thymio, the goal, and the variables used for the controller :

<img src="./images/schema_astolfi.jpg" alt="Diagram" width="200">

We chose to design a controller because it makes the Thymio moves more smoothly than advancing segment by segment and making the robot turn on itself between each segment. Indeed it makes a mix of both the wanted translational and rotational velocities to set the motors' speed.

At first we implemented an Astolfi controller, but it calculates the translational speed proportionnaly to the distance to the goal, and therefore slows down as it gets closer to the goal. We wanted the thymio to have a constant speed when following his path, and therefore we made some changes to the controller :

We start from a baseline translational velocity `SPEED`, which for each wheel we will adapt depending on delta_angle, to obtain a curved and smooth trajectory.

```python
v = SPEED                                                  #translational velocity PWM
omega = k_alpha*(delta_angle) - k_beta*(delta_angle+theta) #rotational velocity [rad/s]

#Calculate motor speed
v_ml = (v+omega*L_AXIS) #PWM
v_mr = (v-omega*L_AXIS) #PWM
```

We tuned the controllers parameters by testing.

* **`k_alpha`** : controls how much the robot turns to align itself with the goal. Depends on `SPEED` : The higher the basic speed is, the more reactive the thymio needs to be to changes of directions.
    * **`k_alpha` too low :** the thymio won't change direction fast enough, it will react too slow to follow the path (therefore not following well the shortest path and risking going over global obtacles)
     * **`k_alpha` too high** : the robot will turn to aggressively and overshoot the turns
and k_beta is a damping term (to stabilize the robot's orientation when reaching the goal).

* **`k_beta`** : damping term if the robot oscillates around his wanted direction. The thymio didn't oscillate and adding a `k_beta` different than zero only made it react too slowly (too much damping), so we decided `k_beta` wasn't needed in this context. We left it in the code because it may be useful to add a damping term in other situations, if the robot's speed was way higher for example, so the code can be easily adapted to other contexts.



## 5.c Local avoidance (braitenberg) :

At each iterations, we call `check_obstacle(prox_values)` to check if the thymio is confronted to a local obstacle. If so the robot will keep applying braitenberg's logic while it can see the obstacle.

`check_obstacle(prox_values)` : check if there is an obstacle based on the maximum proximity sensor value

`avoid_obstacle(prox_values)` : apply braitenberg's logic, the motors' speed is proportional to the proximity sensors values

```python
braitenberg = [-5/300, -20/300, 30/300, 21/300, 6/300] #left to right
v_mr, v_ml = SPEED, SPEED
for i in range (len(prox_values)) :
    v_ml -= braitenberg[i] * prox_values[i]
    v_mr += braitenberg[i] * prox_values[i]
```
We tuned braitenberg's parameters by testing. The values linked to the proximity sensors closer to the center are higher than those linked to the sensors near the edge of the robot because the thymio needs to turn faster to avoid an obstacle directly in front of him than an obstacle more shifted to his left or right.


## Assumptions

- There should always be at least one path from robot to goal, global obstacles cannot completly block the robot and the goal
- Local obstacles should be placed in a way such that during local avoidance maneuvres the robot stays inside the arena and doesn't collide with global obstacles
- Goal and global obstacles should remain fixed during a run and detectable by their color
- Kalman:
    - System is LTI (linear time invariant)
    - Errors are zero mean Gaussians, with known covariance
    - Process and Measurement noise are independant
    - Markovian, $x_t$ only depends on $x_{t-1}$

# PROJECT :